In [17]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from keras import layers
import utils as ut
import numpy as np
from typing import Dict, Optional, Tuple

In [18]:
def get_model():
    model = tf.keras.models.Sequential(
        [
            layers.Input(shape=(8,)),  
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),  
            layers.Dense(1)  
        ]
    )
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
    return model

In [19]:
class CifarClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test):
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""

        # Update local model parameters
        self.model.set_weights(parameters)

        # Get hyperparameters for this round
        batch_size: int = config["batch_size"]
        epochs: int = config["local_epochs"]

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
        )

        # Return updated model parameters, number of examples trained, and results
        parameters_prime = self.model.get_weights()
        num_examples_train = len(self.x_train)
        results = {
            "mse": history.history["mse"][-1],  # Use the last epoch's MSE
            "mae": history.history["mae"][-1],  # Use the last epoch's MAE
            "val_mse": history.history["val_mse"][-1],  # Use the last epoch's validation MSE
            "val_mae": history.history["val_mae"][-1],  # Use the last epoch's validation MAE
        }

        return parameters_prime, num_examples_train, results

    def get_parameters(self):
        """Get the current parameters of the local model."""
        return self.model.get_weights()

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""
        self.model.set_weights(parameters)
        loss = tf.keras.losses.mean_squared_error(self.y_test, self.model.predict(self.x_test)).numpy().mean().item()
        mae = tf.keras.metrics.mean_absolute_error(self.y_test, self.model.predict(self.x_test)).numpy().mean().item()
        print("*************LOSS******************",loss)
        print("************MAE********************",mae)
        return loss, len(self.x_train), {"mae": mae}

    def get_weights(self):
        """Get the current weights of the local model."""
        return self.model.get_weights()

    def set_weights(self, weights):
        """Set the weights of the local model."""
        self.model.set_weights(weights)

In [20]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(0,4,2)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8080",
client=client,
)

INFO flwr 2023-11-23 01:51:25,664 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:51:25,684 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:51:25,686 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/50
26/26 [==============================] - 1s 11ms/step - loss: 4.0173 - mae: 1.6396 - mse: 4.0173 - val_loss: 2.2442 - val_mae: 1.1111 - val_mse: 2.2442
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 1.4311 - mae: 0.9102 - mse: 1.4311 - val_loss: 1.1819 - val_mae: 0.8989 - val_mse: 1.1819
Epoch 3/50
26/26 [==============================] - 0s 4ms/step - loss: 1.0683 - mae: 0.8363 - mse: 1.0683 - val_loss: 1.0102 - val_mae: 0.7971 - val_mse: 1.0102
Epoch 4/50
26/26 [==============================] - 0s 3ms/step - loss: 0.9415 - mae: 0.7695 - mse: 0.9415 - val_loss: 0.8997 - val_mae: 0.7703 - val_mse: 0.8997
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 0.8353 - mae: 0.7293 - mse: 0.8353 - val_loss: 0.7858 - val_mae: 0.6972 - val_mse: 0.7858
Epoch 6/50
26/26 [==============================] - 0s 3ms/step - loss: 0.7210 - mae: 0.6660 - mse: 0.7210 - val_loss: 0.6659 - val_mae: 0.6350 - val_mse: 0.6659
Epoch 7/50
26/26 [=========

DEBUG flwr 2023-11-23 01:53:46,383 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 01:53:46,385 | app.py:215 | Disconnect and shut down


In [21]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(0,4,2)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8081",
client=client,
)

INFO flwr 2023-11-23 01:55:05,465 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:55:05,484 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:55:05,486 | connection.py:42 | ChannelConnectivity.READY


129/129 [==============================] - 0s 1ms/step
*************LOSS****************** 2.2793779373168945
************MAE******************** 1.171873927116394
Epoch 1/100
26/26 [==============================] - 1s 13ms/step - loss: 0.2958 - mae: 0.3699 - mse: 0.2958 - val_loss: 0.2928 - val_mae: 0.3675 - val_mse: 0.2928
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2677 - mae: 0.3519 - mse: 0.2677 - val_loss: 0.2892 - val_mae: 0.3697 - val_mse: 0.2892
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2765 - mae: 0.3641 - mse: 0.2765 - val_loss: 0.2792 - val_mae: 0.3672 - val_mse: 0.2792
Epoch 4/100
26/26 [==============================] - 0s 4ms/step - loss: 0.2604 - mae: 0.3495 - mse: 0.2604 - val_loss: 0.2921 - val_mae: 0.3788 - val_mse: 0.2921
Epoch 5/100
26/26 [==============================] - 0s 4ms/step - loss: 0.2648 - mae: 0.3535 - mse: 0.2648 - val_loss: 0.3023 - val_mae: 0.3699 - val_mse: 0.3023
Epoch 6/100
26/26 [=

DEBUG flwr 2023-11-23 01:57:16,660 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 01:57:16,661 | app.py:215 | Disconnect and shut down


 2.3278539180755615
************MAE******************** 1.1794847249984741


In [22]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(-2,4,2)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8082",
client=client,
)

INFO flwr 2023-11-23 02:00:00,924 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 02:00:00,944 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 02:00:00,947 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/50
26/26 [==============================] - 1s 11ms/step - loss: 1.3995 - mae: 0.7533 - mse: 1.3995 - val_loss: 0.5172 - val_mae: 0.5173 - val_mse: 0.5172
Epoch 2/50
26/26 [==============================] - 0s 4ms/step - loss: 0.4464 - mae: 0.4777 - mse: 0.4464 - val_loss: 0.4271 - val_mae: 0.4705 - val_mse: 0.4271
Epoch 3/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3920 - mae: 0.4457 - mse: 0.3920 - val_loss: 0.3847 - val_mae: 0.4560 - val_mse: 0.3847
Epoch 4/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3539 - mae: 0.4241 - mse: 0.3539 - val_loss: 0.3578 - val_mae: 0.4325 - val_mse: 0.3578
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3384 - mae: 0.4109 - mse: 0.3384 - val_loss: 0.3455 - val_mae: 0.4230 - val_mse: 0.3455
Epoch 6/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3245 - mae: 0.4046 - mse: 0.3245 - val_loss: 0.3332 - val_mae: 0.4143 - val_mse: 0.3332
Epoch 7/50
26/26 [=========

DEBUG flwr 2023-11-23 02:02:12,515 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 02:02:12,515 | app.py:215 | Disconnect and shut down


*************LOSS****************** 2.4310128688812256
************MAE******************** 1.2073485851287842


In [ ]:
print(history)

NameError: name 'histories' is not defined